# Desafío

Zoop ha seleccionado 2 de los visuales creados para formar parte del informe con la narración de datos de la empresa. Se ha definido que se utilizarán los visuales con el facturamiento por región en 2023 y el de personas pre-registradas en Zoop Pay. Refinaremos ambos visuales siguiendo las instrucciones a continuación y la paleta de colores de la e-commerce, alterándolos con la ayuda de ChatGPT:

## Desafío 1:

Construye y ejecuta un prompt en ChatGPT para refinar el visual que presenta el facturamiento de las ventas por región. Las instrucciones para el refinamiento son las siguientes:

- Fondo de la figura y del área del gráfico en el color GRIS_1
- Color de las columnas en AQUA_1 (si deseas destacar solo una, puedes recurrir a AQUA_3, por ejemplo)
- Tamaño de la figura de (16,9)
- Cambia el título de la figura a un tamaño de 18 y color GRIS_5, ajusta las etiquetas y rótulos del eje x e y, cuando existan, a un tamaño 12 y color AZUL_1
- Elimina el máximo de bordes posible y ajusta los datos para que se presenten en las barras en orden de millones, con color AZUL_1 y tamaño 12
**Observación 1:** Todas las variables que representan los colores en las instrucciones están en una celda en la pestaña Paleta de Colores dentro del notebook Desafíos de la Clase 04.

**Observación 2:** Es probable que al ejecutar los prompts, ChatGPT sugiera colores para las variables y modificaciones en los códigos que llaman al DataFrame deseado. Presta mucha atención para eliminar la nueva definición de colores y ajustar el DataFrame según lo que has hecho en actividades anteriores.

## Desafío 2:

Construye y ejecuta un prompt en ChatGPT para refinar el visual que presenta la proporción de personas pre-registradas en Zoop Pay. Las instrucciones para el refinamiento son las siguientes:

- Fondo de la figura y del área del gráfico en el color GRIS_1.
- Color de las porciones: AZUL_1 para "Sí" y GRIS_3 para "No".
- Tamaño de la figura de (6,6).
- Cambia el título de la figura a un tamaño de 18 y color GRIS_5.
- Añade etiquetas en las porciones con tamaño 12 y los porcentajes también con tamaño 12 y en negrita. Los porcentajes deben estar en color BLANCO y las etiquetas de las porciones en el mismo color que las porciones.
- Ajusta la rosca para que su ancho sea de 0.4 y los porcentajes de las porciones estén en la posición 0.8 en relación al centro.
- Evita explosiones de porciones y efectos que dificulten el análisis del gráfico.